# Programmatic CDL Composition Tutorial

This interactive notebook demonstrates how to build Control Description Language (CDL) control systems programmatically using Python and Pydantic models.

## What You'll Learn

1. **Creating Elementary Blocks** - Build individual control components
2. **Defining Connectors** - Set up typed inputs and outputs with units
3. **Setting Parameters** - Configure controller gains and limits
4. **Composing Systems** - Wire blocks together into complete systems
5. **Exporting to CDL-JSON** - Save your work in standard format
6. **Validation** - Ensure correctness before deployment

## Why Programmatic Composition?

**Benefits over hand-writing JSON:**
- ✅ **Type Safety** - Pydantic catches errors at creation time
- ✅ **Code Completion** - IDE support for all fields
- ✅ **Reusability** - Create functions for common patterns
- ✅ **Version Control** - Git-friendly Python code
- ✅ **Testing** - Unit test your control logic
- ✅ **Dynamic Generation** - Build controls based on configuration

Let's get started!

---

## Setup: Import Libraries

In [1]:
# Standard library
import json
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import Python CDL models
from python_cdl.models.blocks import CompositeBlock, ElementaryBlock
from python_cdl.models.connectors import RealInput, RealOutput, BooleanInput, BooleanOutput
from python_cdl.models.connections import Connection
from python_cdl.models.parameters import Parameter
from python_cdl.models.types import CDLTypeEnum
from python_cdl.validators import BlockValidator, GraphValidator
from python_cdl.parser import CDLParser

print("✓ All imports successful!")
print(f"✓ Python CDL loaded from: {project_root}")

✓ All imports successful!
✓ Python CDL loaded from: /Users/acedrew/aceiot-projects/python-cdl


---

## Part 1: Creating Elementary Blocks

Elementary blocks are the building blocks of control systems. Let's create a simple PI controller.

In [2]:
# Create a PI controller block
pi_controller = ElementaryBlock(
    name="TemperatureController",
    block_type="Buildings.Controls.OBC.CDL.Continuous.LimPID",
    category="elementary",
    description="PI controller for temperature control",
    inputs=[
        RealInput(
            name="u_s",
            type=CDLTypeEnum.REAL,
            unit="K",
            quantity="ThermodynamicTemperature",
            description="Setpoint temperature"
        ),
        RealInput(
            name="u_m",
            type=CDLTypeEnum.REAL,
            unit="K",
            quantity="ThermodynamicTemperature",
            description="Measured temperature"
        ),
    ],
    outputs=[
        RealOutput(
            name="y",
            type=CDLTypeEnum.REAL,
            unit="1",
            min=0.0,
            max=1.0,
            description="Control signal (0-1)"
        ),
    ],
    parameters=[
        Parameter(
            name="k",
            type=CDLTypeEnum.REAL,
            value=0.5,
            unit="1",
            description="Proportional gain"
        ),
        Parameter(
            name="Ti",
            type=CDLTypeEnum.REAL,
            value=60.0,
            unit="s",
            quantity="Time",
            description="Integral time constant"
        ),
    ],
)

print(f"Created block: {pi_controller.name}")
print(f"  Type: {pi_controller.block_type}")
print(f"  Inputs: {len(pi_controller.inputs)}")
print(f"  Outputs: {len(pi_controller.outputs)}")
print(f"  Parameters: {len(pi_controller.parameters)}")
print(f"\nParameters:")
for param in pi_controller.parameters:
    print(f"  • {param.name} = {param.value} {param.unit}")

Created block: TemperatureController
  Type: Buildings.Controls.OBC.CDL.Continuous.LimPID
  Inputs: 2
  Outputs: 1
  Parameters: 2

Parameters:
  • k = 0.5 1
  • Ti = 60.0 s


### Key Concepts

**ElementaryBlock** represents a single control component:
- `name` - Unique identifier for the block
- `block_type` - CDL type reference (from Buildings library)
- `category` - "elementary" for basic blocks
- `inputs` - Typed input connectors
- `outputs` - Typed output connectors
- `parameters` - Configuration values

**Type Safety**: Pydantic validates all fields automatically!

---

## Part 2: Exporting a Single Block

Let's export our PI controller to CDL-JSON format.

In [3]:
# Export to JSON
pi_json = pi_controller.model_dump(mode='json', exclude_none=True)

print("CDL-JSON Export:")
print("=" * 70)
print(json.dumps(pi_json, indent=2)[:500] + "...")
print("\n✓ Export successful!")

CDL-JSON Export:
{
  "name": "TemperatureController",
  "block_type": "Buildings.Controls.OBC.CDL.Continuous.LimPID",
  "parameters": [
    {
      "name": "k",
      "type": "Real",
      "value": 0.5,
      "unit": "1",
      "description": "Proportional gain"
    },
    {
      "name": "Ti",
      "type": "Real",
      "value": 60.0,
      "quantity": "Time",
      "unit": "s",
      "description": "Integral time constant"
    }
  ],
  "constants": [],
  "inputs": [
    {
      "name": "u_s",
      "type": "R...

✓ Export successful!


### Pydantic's `model_dump()` Method

The `model_dump()` method converts Pydantic models to dictionaries:
- `mode='json'` - Use JSON-compatible types
- `exclude_none=True` - Skip optional fields that are None

Result is a standard Python dict that can be serialized to JSON.

---

## Part 3: Creating Helper Functions

For reusability, let's create functions that generate common block types.

In [4]:
def create_constant_source(name: str, value: float, unit: str = "1") -> ElementaryBlock:
    """Create a constant value source block."""
    return ElementaryBlock(
        name=name,
        block_type="Buildings.Controls.OBC.CDL.Continuous.Sources.Constant",
        category="elementary",
        description=f"Constant value: {value} {unit}",
        outputs=[
            RealOutput(
                name="y",
                type=CDLTypeEnum.REAL,
                unit=unit,
                description="Constant output value"
            ),
        ],
        parameters=[
            Parameter(
                name="k",
                type=CDLTypeEnum.REAL,
                value=value,
                unit=unit,
                description="Constant value"
            ),
        ],
    )

def create_gain(name: str, k: float) -> ElementaryBlock:
    """Create a gain (multiplier) block."""
    return ElementaryBlock(
        name=name,
        block_type="Buildings.Controls.OBC.CDL.Continuous.Gain",
        category="elementary",
        description=f"Multiply input by {k}",
        inputs=[
            RealInput(name="u", type=CDLTypeEnum.REAL, description="Input signal"),
        ],
        outputs=[
            RealOutput(name="y", type=CDLTypeEnum.REAL, description="Output signal"),
        ],
        parameters=[
            Parameter(name="k", type=CDLTypeEnum.REAL, value=k, description="Gain value"),
        ],
    )

def create_limiter(name: str, y_min: float, y_max: float) -> ElementaryBlock:
    """Create a limiter block."""
    return ElementaryBlock(
        name=name,
        block_type="Buildings.Controls.OBC.CDL.Continuous.Limiter",
        category="elementary",
        description=f"Limit output between {y_min} and {y_max}",
        inputs=[
            RealInput(name="u", type=CDLTypeEnum.REAL, description="Input signal"),
        ],
        outputs=[
            RealOutput(
                name="y",
                type=CDLTypeEnum.REAL,
                min=y_min,
                max=y_max,
                description="Limited output signal"
            ),
        ],
        parameters=[
            Parameter(name="uMin", type=CDLTypeEnum.REAL, value=y_min),
            Parameter(name="uMax", type=CDLTypeEnum.REAL, value=y_max),
        ],
    )

# Test our helper functions
setpoint = create_constant_source("Setpoint", 21.0, "degC")
gain_block = create_gain("ErrorGain", 2.0)
limiter = create_limiter("OutputLimiter", 0.0, 1.0)

print("Created helper blocks:")
print(f"  • {setpoint.name}: {setpoint.description}")
print(f"  • {gain_block.name}: {gain_block.description}")
print(f"  • {limiter.name}: {limiter.description}")

Created helper blocks:
  • Setpoint: Constant value: 21.0 degC
  • ErrorGain: Multiply input by 2.0
  • OutputLimiter: Limit output between 0.0 and 1.0


### Helper Function Benefits

Creating helper functions:
- **Reduces boilerplate** - Less repetitive code
- **Enforces consistency** - Same pattern every time
- **Simplifies maintenance** - Change once, apply everywhere
- **Documents intent** - Function name explains purpose

---

## Part 4: Building a Composite System

Now let's wire multiple blocks together into a complete control system.

In [5]:
# Create component blocks
setpoint_source = create_constant_source("SetpointSource", 294.15, "K")  # 21°C
controller = ElementaryBlock(
    name="PIController",
    block_type="Buildings.Controls.OBC.CDL.Continuous.LimPID",
    category="elementary",
    inputs=[
        RealInput(name="u_s", type=CDLTypeEnum.REAL, unit="K"),
        RealInput(name="u_m", type=CDLTypeEnum.REAL, unit="K"),
    ],
    outputs=[
        RealOutput(name="y", type=CDLTypeEnum.REAL, unit="1", min=0.0, max=1.0),
    ],
    parameters=[
        Parameter(name="k", type=CDLTypeEnum.REAL, value=0.8),
        Parameter(name="Ti", type=CDLTypeEnum.REAL, value=120.0, unit="s"),
    ],
)
output_limiter = create_limiter("ValveLimiter", 0.0, 1.0)

# Create composite system
control_system = CompositeBlock(
    name="BasicTemperatureControl",
    block_type="TemperatureControlSystem",
    category="composite",
    description="Basic temperature control with PI controller and limiter",
    inputs=[
        RealInput(
            name="temperature",
            type=CDLTypeEnum.REAL,
            unit="K",
            description="Measured room temperature"
        ),
    ],
    outputs=[
        RealOutput(
            name="valve_position",
            type=CDLTypeEnum.REAL,
            unit="1",
            min=0.0,
            max=1.0,
            description="Heating valve position"
        ),
    ],
    blocks=[setpoint_source, controller, output_limiter],
    connections=[
        # Setpoint to controller
        Connection(
            from_block="SetpointSource",
            from_output="y",
            to_block="PIController",
            to_input="u_s",
            description="Setpoint to controller"
        ),
        # External input to controller
        Connection(
            from_block="temperature",
            from_output="temperature",
            to_block="PIController",
            to_input="u_m",
            description="Temperature measurement"
        ),
        # Controller to limiter
        Connection(
            from_block="PIController",
            from_output="y",
            to_block="ValveLimiter",
            to_input="u",
            description="Control signal"
        ),
        # Limiter to external output
        Connection(
            from_block="ValveLimiter",
            from_output="y",
            to_block="valve_position",
            to_input="valve_position",
            description="Limited valve position"
        ),
    ],
)

print(f"Created composite system: {control_system.name}")
print(f"\nSystem Structure:")
print(f"  Inputs: {len(control_system.inputs)}")
print(f"  Outputs: {len(control_system.outputs)}")
print(f"  Internal Blocks: {len(control_system.blocks)}")
print(f"  Connections: {len(control_system.connections)}")
print(f"\nData Flow:")
for conn in control_system.connections:
    print(f"  {conn.from_block}.{conn.from_output} → {conn.to_block}.{conn.to_input}")

Created composite system: BasicTemperatureControl

System Structure:
  Inputs: 1
  Outputs: 1
  Internal Blocks: 3
  Connections: 4

Data Flow:
  SetpointSource.y → PIController.u_s
  temperature.temperature → PIController.u_m
  PIController.y → ValveLimiter.u
  ValveLimiter.y → valve_position.valve_position


### Composite Block Structure

**CompositeBlock** represents a system of interconnected blocks:
- `blocks` - List of internal elementary/composite blocks
- `connections` - Wiring between blocks and external interfaces
- `inputs` - External inputs to the system
- `outputs` - External outputs from the system

**Connection** represents a wire:
- `from_block` - Source block name (or input name)
- `from_output` - Source output port name
- `to_block` - Destination block name (or output name)
- `to_input` - Destination input port name

---

## Part 5: Validation

Before exporting, let's validate our system.

In [6]:
# Validate block structure
block_validator = BlockValidator()
result = block_validator.validate(control_system)

if hasattr(result, 'is_valid'):
    is_valid = result.is_valid
    errors = result.errors if hasattr(result, 'errors') else []
else:
    is_valid, errors = result

print("Block Structure Validation:")
if is_valid:
    print("  ✓ Valid")
else:
    print(f"  ✗ Errors: {errors}")

# Validate connection graph
graph_validator = GraphValidator()
result = graph_validator.validate(control_system)

if hasattr(result, 'is_valid'):
    is_valid = result.is_valid
    errors = result.errors if hasattr(result, 'errors') else []
else:
    is_valid, errors = result

print("\nConnection Graph Validation:")
if is_valid:
    print("  ✓ Valid (no cycles detected)")
else:
    print(f"  ⚠ Warnings: {errors}")

print("\n✅ System is ready for export!")

Block Structure Validation:
  ✓ Valid

Connection Graph Validation:
  ✓ Valid (no cycles detected)

✅ System is ready for export!


### Validation Checks

**BlockValidator** checks:
- All required fields are present
- Block names are unique
- Connector types are valid
- Parameters have correct types

**GraphValidator** checks:
- No circular dependencies
- All connections reference valid blocks
- Connector names exist
- Type compatibility (future)

---

## Part 6: Export to CDL-JSON

Export our validated system to CDL-JSON format.

In [7]:
# Export to JSON
output_dir = Path("notebook_output")
output_dir.mkdir(exist_ok=True)
output_file = output_dir / "basic_temp_control.json"

cdl_json = control_system.model_dump(mode='json', exclude_none=True)

with open(output_file, 'w') as f:
    json.dump(cdl_json, f, indent=2)

file_size = output_file.stat().st_size
print(f"✓ Exported to {output_file}")
print(f"  File size: {file_size:,} bytes")

# Show snippet
print(f"\nJSON Structure (first 600 chars):")
print("=" * 70)
json_str = json.dumps(cdl_json, indent=2)
print(json_str[:600] + "...")

✓ Exported to notebook_output/basic_temp_control.json
  File size: 3,621 bytes

JSON Structure (first 600 chars):
{
  "name": "BasicTemperatureControl",
  "block_type": "TemperatureControlSystem",
  "parameters": [],
  "constants": [],
  "inputs": [
    {
      "name": "temperature",
      "type": "Real",
      "unit": "K",
      "description": "Measured room temperature",
      "direction": "input"
    }
  ],
  "outputs": [
    {
      "name": "valve_position",
      "type": "Real",
      "unit": "1",
      "description": "Heating valve position",
      "direction": "output"
    }
  ],
  "equations": [],
  "description": "Basic temperature control with PI controller and limiter",
  "category": "composite...


---

## Part 7: Round-Trip Testing

Verify we can re-import the exported JSON.

In [8]:
# Re-import the JSON
parser = CDLParser()
with open(output_file) as f:
    json_data = json.load(f)

reimported = parser.parse(json_data)

print("Round-Trip Test:")
print(f"  Original name: {control_system.name}")
print(f"  Reimported name: {reimported.name}")
print(f"  Match: {control_system.name == reimported.name} ✓")

print(f"\n  Original blocks: {len(control_system.blocks)}")
print(f"  Reimported blocks: {len(reimported.blocks)}")
print(f"  Match: {len(control_system.blocks) == len(reimported.blocks)} ✓")

print(f"\n  Original connections: {len(control_system.connections)}")
print(f"  Reimported connections: {len(reimported.connections)}")
print(f"  Match: {len(control_system.connections) == len(reimported.connections)} ✓")

print("\n✅ Round-trip successful! Export/import works correctly.")

Round-Trip Test:
  Original name: BasicTemperatureControl
  Reimported name: BasicTemperatureControl
  Match: True ✓

  Original blocks: 3
  Reimported blocks: 3
  Match: True ✓

  Original connections: 4
  Reimported connections: 4
  Match: True ✓

✅ Round-trip successful! Export/import works correctly.


---

## Part 8: Interactive Exercise - Build Your Own Controller

Now it's your turn! Modify the code below to create a different control system.

In [9]:
# Exercise: Create a simple on/off controller
#
# Requirements:
# 1. Compare temperature to setpoint
# 2. Use hysteresis block for on/off decision
# 3. Output boolean control signal
#
# Hints:
# - Use Buildings.Controls.OBC.CDL.Continuous.Add for comparison (k2=-1)
# - Use Buildings.Controls.OBC.CDL.Continuous.Hysteresis for on/off logic
# - BooleanOutput for the control signal

# TODO: Your code here!

print("Exercise: Implement your own on/off controller!")
print("Uncomment and complete the code above.")

Exercise: Implement your own on/off controller!
Uncomment and complete the code above.


---

## Part 9: Advanced Pattern - Reusable Controller Factory

Let's create a factory function that generates customized control systems.

In [10]:
from typing import Dict, Any

def create_pi_temperature_system(
    name: str,
    setpoint_celsius: float,
    kp: float = 0.5,
    ti: float = 60.0,
    **kwargs
) -> CompositeBlock:
    """Factory function for PI temperature control systems.
    
    Args:
        name: System name
        setpoint_celsius: Setpoint in °C
        kp: Proportional gain
        ti: Integral time in seconds
        **kwargs: Additional options
    
    Returns:
        CompositeBlock ready to use
    """
    setpoint_kelvin = setpoint_celsius + 273.15
    
    # Create blocks
    setpoint = create_constant_source(f"{name}_Setpoint", setpoint_kelvin, "K")
    
    controller = ElementaryBlock(
        name=f"{name}_Controller",
        block_type="Buildings.Controls.OBC.CDL.Continuous.LimPID",
        category="elementary",
        inputs=[
            RealInput(name="u_s", type=CDLTypeEnum.REAL, unit="K"),
            RealInput(name="u_m", type=CDLTypeEnum.REAL, unit="K"),
        ],
        outputs=[
            RealOutput(name="y", type=CDLTypeEnum.REAL, unit="1", min=0.0, max=1.0),
        ],
        parameters=[
            Parameter(name="k", type=CDLTypeEnum.REAL, value=kp),
            Parameter(name="Ti", type=CDLTypeEnum.REAL, value=ti, unit="s"),
        ],
    )
    
    # Create composite
    system = CompositeBlock(
        name=name,
        block_type="PI_TemperatureControl",
        category="composite",
        description=f"PI temperature control for {setpoint_celsius}°C setpoint",
        inputs=[
            RealInput(name="temperature", type=CDLTypeEnum.REAL, unit="K"),
        ],
        outputs=[
            RealOutput(name="control_signal", type=CDLTypeEnum.REAL, unit="1", min=0.0, max=1.0),
        ],
        blocks=[setpoint, controller],
        connections=[
            Connection(
                from_block=f"{name}_Setpoint",
                from_output="y",
                to_block=f"{name}_Controller",
                to_input="u_s"
            ),
            Connection(
                from_block="temperature",
                from_output="temperature",
                to_block=f"{name}_Controller",
                to_input="u_m"
            ),
            Connection(
                from_block=f"{name}_Controller",
                from_output="y",
                to_block="control_signal",
                to_input="control_signal"
            ),
        ],
    )
    
    return system

# Use the factory to create multiple controllers
office_control = create_pi_temperature_system("OfficeControl", setpoint_celsius=21.0, kp=0.6, ti=90.0)
lab_control = create_pi_temperature_system("LabControl", setpoint_celsius=20.0, kp=0.8, ti=120.0)
lobby_control = create_pi_temperature_system("LobbyControl", setpoint_celsius=19.0, kp=0.4, ti=60.0)

print("Created 3 customized controllers using factory:")
print(f"  • {office_control.name}: {office_control.description}")
print(f"  • {lab_control.name}: {lab_control.description}")
print(f"  • {lobby_control.name}: {lobby_control.description}")

print("\n💡 Factory pattern allows:")
print("  - Consistent structure across controllers")
print("  - Easy customization via parameters")
print("  - Rapid prototyping")
print("  - Configuration-driven generation")

Created 3 customized controllers using factory:
  • OfficeControl: PI temperature control for 21.0°C setpoint
  • LabControl: PI temperature control for 20.0°C setpoint
  • LobbyControl: PI temperature control for 19.0°C setpoint

💡 Factory pattern allows:
  - Consistent structure across controllers
  - Easy customization via parameters
  - Rapid prototyping
  - Configuration-driven generation


---

## Summary

### What We Learned

✅ **Creating Elementary Blocks** - Built PI controllers, gains, limiters  
✅ **Defining Connectors** - Typed inputs/outputs with units  
✅ **Setting Parameters** - Configured controller gains  
✅ **Composing Systems** - Wired blocks into complete systems  
✅ **Exporting CDL-JSON** - Used `model_dump()` for serialization  
✅ **Validation** - Checked structure and connections  
✅ **Round-Trip Testing** - Verified export/import  
✅ **Factory Pattern** - Created reusable generators  

### Key Takeaways

1. **Type Safety** - Pydantic catches errors early
2. **Reusability** - Helper functions reduce boilerplate
3. **Validation** - Multiple levels ensure correctness
4. **Interoperability** - Standard CDL-JSON format
5. **Testability** - Unit test control logic before deployment

### Next Steps

- Explore the `simple_temperature_controller.py` example
- Try the `room_control_system.py` advanced example
- Build your own control systems
- Create libraries of reusable controllers
- Deploy to real building systems

---

**Questions or feedback?** Open an issue on the Python CDL GitHub repository.

*Tutorial created with Python CDL v0.1.0*